# Init EDA

purpose of NB: try to understand the data itself

Determine a plan of attack for a baseline approach + next steps on how to clean the dataset and then get into feature engineering


In [1]:
%pip install -r ../requirements.txt

c:\Users\mlar5\OneDrive\Desktop\Code Folder Undergrad\Python Projects\IRL projects\Kaggle\Insurance_Regression
  Obtaining dependency information for pandas==2.0.3 from https://files.pythonhosted.org/packages/9e/71/756a1be6bee0209d8c0d8c5e3b9fc72c00373f384a4017095ec404aec3ad/pandas-2.0.3-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.6 MB 1.8 MB/s eta 0:00:06
   -- ------------------------------------- 0.7/10.6 MB 6.3 MB/s eta 0:00:02
   --------- ------------------------------ 2.5/10.6 MB 15.8 MB/s eta 0:00:01
   ------------ --------------------------- 3.3/10.6 MB 19.1 MB/s eta 0:00:01
   ----------------------- ---------------- 6.2/10.6 MB 24.7 MB/s eta 0:00:01
   ------------------------ --------------- 6.6/10.6 MB 24.8 MB/s eta 0:00:01
   ------------------------------------- -- 9.9/10.6 MB 28.7 MB/s eta 


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# import/load data


In [1]:
import pandas as pd

In [73]:
train_df = pd.read_csv("../data/raw_dataset/train.csv")

# Save base dataset
This serves as the main dataset used to build the future refined versions of our datasets. Below are the basic changes made:
- drop id col
- drop duplicates
- convert `Policy Start Date` to `Policy Duration Mins`
- convert string data to categorical type


In [74]:
#remove id col and duplicates
train_df = train_df.drop(columns=['id'])
train_df=train_df.drop_duplicates()
#convert start date to a timespan in mins
train_df['Policy Start Date'] = pd.to_datetime(train_df['Policy Start Date'])
train_df['Policy Duration Mins'] = ((pd.Timestamp.now() - train_df['Policy Start Date']).dt.total_seconds())/60
train_df = train_df.drop(columns=['Policy Start Date'])
# convert string cols to categorical for future encoding
categorical_cols = [train_df.columns[i] for i, x in enumerate(train_df.dtypes) if x == 'object']
# Create new columns as categorical and perform one-hot
train_df[categorical_cols] = train_df[categorical_cols].astype('category')

train_df.to_csv('../data/train_base.csv')

# General data info

This section contains insights with shape, null counts, data types

I see many rows of data, which may make model training on full dataset time costly. Will likely want to work on a statistically proportional subset of the dataset for the first few iterations of training

In [5]:
train_df.shape

(1200000, 21)

Here's the breakdown of datatypes:

1 continuous feature (health score)

9 discrete features (8 float types and 1 object -- technically start date is `object` but can be converted to int)

10 categorical


about half of the columns have empty missing data
- this will be addressed on a row-by-row basis

In [16]:
train_df.head(1)

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0


In [27]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 20 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Age                   1181295 non-null  float64
 1   Gender                1200000 non-null  object 
 2   Annual Income         1155051 non-null  float64
 3   Marital Status        1181471 non-null  object 
 4   Number of Dependents  1090328 non-null  float64
 5   Education Level       1200000 non-null  object 
 6   Occupation            841925 non-null   object 
 7   Health Score          1125924 non-null  float64
 8   Location              1200000 non-null  object 
 9   Policy Type           1200000 non-null  object 
 10  Previous Claims       835971 non-null   float64
 11  Vehicle Age           1199994 non-null  float64
 12  Credit Score          1062118 non-null  float64
 13  Insurance Duration    1199999 non-null  float64
 14  Policy Start Date     1200000 non-

In [28]:
train_df.describe()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,1.181295e+06,1.155051e+06,1.090328e+06,1.125924e+06,835971.000000,1.199994e+06,1.062118e+06,1.199999e+06,1.200000e+06
mean,4.114556e+01,3.274522e+04,2.009934e+00,2.561391e+01,1.002689,9.569889e+00,5.929244e+02,5.018219e+00,1.102545e+03
std,1.353995e+01,3.217951e+04,1.417338e+00,1.220346e+01,0.982840,5.776189e+00,1.499819e+02,2.594331e+00,8.649989e+02
min,1.800000e+01,1.000000e+00,0.000000e+00,2.012237e+00,0.000000,0.000000e+00,3.000000e+02,1.000000e+00,2.000000e+01
25%,3.000000e+01,8.001000e+03,1.000000e+00,1.591896e+01,0.000000,5.000000e+00,4.680000e+02,3.000000e+00,5.140000e+02
50%,4.100000e+01,2.391100e+04,2.000000e+00,2.457865e+01,1.000000,1.000000e+01,5.950000e+02,5.000000e+00,8.720000e+02
75%,5.300000e+01,4.463400e+04,3.000000e+00,3.452721e+01,2.000000,1.500000e+01,7.210000e+02,7.000000e+00,1.509000e+03
max,6.400000e+01,1.499970e+05,4.000000e+00,5.897591e+01,9.000000,1.900000e+01,8.490000e+02,9.000000e+00,4.999000e+03


In [7]:
train_df.isnull().sum()

Age                      18705
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score             74076
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Policy Start Date            0
Customer Feedback        77824
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64

# Plan of attack

0. save the slightly altered dataset (`base_train.csv`)

###  Baseline model-based approach
1. make a baseline dataset (no processing)
    1. remove rows with nulls    
2. Use a few baseline models (no hyperparam tuning)
    1. Linear Regression
    2. Random Forest
    3. SVM
3. Compare against best heuristic approach (always predict median insurance value)
